In [1]:
import time
import random, string
from tqdm import tnrange

dataLen = 10000
keyLen = 100
valueLen = 100

Insert test

In [2]:
import pymysql

class DataToMysql:
    def __init__(self, host, user, passwd, db, port):
        try:
            self.conn = pymysql.connect(host=host, user=user, passwd=passwd, db=db,
                                        port=port, charset='utf8')  # 鏈接數據庫
            self.cursor = self.conn.cursor()
        except pymysql.Error as e:
            raise e
            
    def getConn(self):
        return self.conn
    
    def dropTable(self, table_name):
        self.cursor.execute("DROP TABLE IF EXISTS %s" % table_name)

    def write(self, table_name, info_dict):
        """
        根據table_name與info自動生成建表語句和insert插入語句
        :param table_name: 數據需要寫入的表名
        :param info_dict: 需要寫入的內容，類型為字典
        :return:
        """
        sql_key = ''  # 數據庫行字段
        sql_value = ''  # 數據庫值
        for key in info_dict.keys():  # 生成insert插入語句
            sql_value = (sql_value + '"' + pymysql.escape_string(info_dict[key]) + '"' + ',')
            sql_key = sql_key + ' ' + key + ','

        try:
            self.cursor.execute(
                "INSERT INTO %s (%s) VALUES (%s)" % (table_name, sql_key[:-1], sql_value[:-1]))
            self.conn.commit()  # 提交當前事務
        except pymysql.Error as e:
            if str(e).split(',')[0].split('(')[1] == "1146":  # 當表不存在時，生成建表語句並建表
                sql_key_str = ''  # 用於數據庫創建語句
                columnStyle = ' text'  # 數據庫字段類型
                for key in info_dict.keys():
                    sql_key_str = sql_key_str + ' ' + key + columnStyle + ','
                self.cursor.execute("CREATE TABLE %s (%s)" % (table_name, sql_key_str[:-1]))
                self.cursor.execute("INSERT INTO %s (%s) VALUES (%s)" %
                                    (table_name, sql_key[:-1], sql_value[:-1]))
                self.conn.commit()  # 提交當前事務
            else:
                raise
                
    def read(self, table_name, where):
        self.cursor.execute("SELECT * FROM %s WHERE %s" % (table_name, where))
        return self.cursor.fetchall()

In [3]:
col = [''.join(random.choice(string.ascii_letters) for _ in range(10)) for _ in range(keyLen)]
result = []

mysql = DataToMysql(host='140.124.181.78',port=30513,user='wei840222',passwd='j04rmp4jo3',db='database')
mysql.dropTable('test')
for i in tnrange(dataLen):
    data = {'id':str(i)}
    for x in range(keyLen): 
        data[col[x]] = ''.join(random.choice(string.ascii_letters) for _ in range(valueLen))
    start = time.time()
    mysql.write('test', data)
    end = time.time()
    result.append(end-start)
mysql.getConn().close()

InternalError: (1130, "Host 'healthcare.csie.ntut.edu.tw' is not allowed to connect to this MySQL server")

output result to csv

In [ ]:
with open('mysql/longhorn_10000_insert.csv', 'w') as f:
    f.write('time\n')
    for r in result:
        f.write(str(r)+'\n')

Download test

In [ ]:
result = []

mysql = DataToMysql(host='140.124.181.78',port=30513,user='wei840222',passwd='j04rmp4jo3',db='database')
mysql.dropTable('test')
for i in tnrange(dataLen):
    start = time.time()
    mysql.read('test', 'id = %d' % i)
    end = time.time()
    result.append(end-start)
mysql.getConn().close()

In [ ]:
with open('mysql/longhorn_10000_read.csv', 'w') as f:
    f.write('time\n')
    for r in result:
        f.write(str(r)+'\n') 